In [ ]:
!pip install pyspark 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 20.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=274c88c46d7255f005e185403c403b6107074cffe2fa8f32830a98a1300cbf82
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [17]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("clean_2.csv",index_col=0,nrows=5000)

df1 = df

In [3]:
import numpy as np
num_rows = df.shape[0]
random_rows = np.random.choice(num_rows, size=int(num_rows*0.01), replace=False)
subset_df = df.iloc[random_rows, :]

In [4]:
features = ['BusinessYear', 'StateCode', 'SourceName', 'RatingAreaId', 'Tobacco', 'Age', 'PrimarySubscriberAndThreeOrMoreDependents','tobacco_rate']
target = 'PlanId'
df.head()

,BusinessYear,StateCode,SourceName,PlanId,RatingAreaId,Tobacco,Age,IndividualRate,PrimarySubscriberAndThreeOrMoreDependents,tobacco_rate
0,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,10,29.00,0.0,29.00
1,2014,AK,HIOS,21989AK0020001,Rating Area 1,No Preference,-5,36.95,1.0,107.61
2,2014,AK,HIOS,21989AK0020001,Rating Area 2,No Preference,-5,36.95,1.0,107.61
3,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,21,32.00,0.0,32.00
4,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,22,32.00,0.0,32.00


In [7]:
from sklearn.preprocessing import LabelEncoder
cols_to_encode = ['SourceName','StateCode','Tobacco','PlanId','RatingAreaId']

les = [LabelEncoder() for _ in range(len(cols_to_encode))]
for i,col in enumerate(cols_to_encode):
    df[col] = les[i].fit_transform(df[col])


In [8]:
df.head(2)

,BusinessYear,StateCode,SourceName,PlanId,RatingAreaId,Tobacco,Age,IndividualRate,PrimarySubscriberAndThreeOrMoreDependents,tobacco_rate
0,2014,0,0,0,0,0,10,29.00,0.0,29.00
1,2014,0,0,1,0,0,-5,36.95,1.0,107.61


In [10]:
X_train, X_test, y_train, y_test= train_test_split(df[features], df[[target]], test_size=0.2, random_state=42)

In [11]:
X_train

,BusinessYear,StateCode,SourceName,RatingAreaId,Tobacco,Age,PrimarySubscriberAndThreeOrMoreDependents,tobacco_rate
4227,2014,0,0,0,0,33,0.0,357.00
4676,2014,0,0,2,0,31,0.0,462.00
800,2014,0,0,2,0,10,0.0,47.06
3671,2014,0,0,0,0,60,0.0,1159.58
4193,2014,0,0,0,0,27,0.0,373.98
...,...,...,...,...,...,...,...,...
4426,2014,0,0,1,0,27,0.0,417.75
466,2014,0,0,1,0,10,0.0,38.46
3092,2014,0,0,1,0,36,0.0,568.38
3772,2014,0,0,1,0,34,0.0,526.00


In [12]:
user_recs = []
for i,user_features in enumerate(X_test.iterrows()):
    similarity_scores = np.array([cosine_similarity(user_features[1].values.reshape(1, -1), item[1].values.reshape(1, -1))[0] for t,item in enumerate(X_train.iterrows())])
    top_indices = np.argsort(similarity_scores.flatten())[-10:]
    user_recs.append({'user': i, 'recommendations': top_indices})


In [ ]:
user_id = 0
recommendations = user_recs[user_id]['recommendations']
print(f"Recommendations for user {user_id}:")
for i, plan_idx in enumerate(recommendations):
  print(les[3].inverse_transform(y_train.iloc[plan_idx]))
  print(les[3].inverse_transform(y_test.iloc[user_id]))



In [19]:
y_pred = []
for user_id in range(len(X_test)):
  recommendations = user_recs[user_id]['recommendations']
  y_pred.append(les[3].inverse_transform(y_train.iloc[recommendations[-1]]))

In [20]:
print(f1_score(les[3].inverse_transform(y_test),y_pred,average='weighted'))

0.7815498011097902


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
